<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v2/blob/main/myProspects_Upside_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/tradevesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Import the function from the script
import myProspects_Upside_Analysis_v2

# Call the function to get the DataFrame
df_prospects_features = myProspects_Upside_Analysis_v2.get_prospects_features_v2()

Run date time (IST): 2024-07-19 17:17:50
-------------------
qualified stocks: 164
with latest results: 152
still star stocks: 91


In [3]:
# avsp
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['AVSP'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Momentum,Today%,FTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
88,JSWHL,6847.0,7487.0,14.56,63.44,UP,-2.09,9.35,56.40,AVSP,BTT,1,0,H,FINANCE
7,AMBUJACEM,677.9,733.0,22.36,70.00,UP,-1.79,8.13,67.21,AVSP,BTT,0,0,H,CEMENT


In [4]:
# av40
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['AV40'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Momentum,Today%,FTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
83,ITC,474.55,576.00,10.88,33.39,NaN,0.91,21.38,19.74,AV40,BTT,1,1,H,FMCG
147,TITAN,3259.00,3854.04,-6.27,-4.41,NaN,-0.10,18.26,12.67,AV40,ATH,1,1,H,JEWELLERY
59,GILLETTE,7683.90,8980.00,15.95,6.23,UP,-3.64,16.87,43.58,AV40,BTT,0,0,H,FMCG
5,AKZOINDIA,2841.55,3300.00,10.95,-11.40,UP,-3.17,16.13,24.29,AV40,NTT,1,1,H,PAINTS
116,PFIZER,5059.35,5855.00,17.09,15.29,UP,-0.49,15.73,32.60,AV40,NTT,1,0,H,PHARMA
117,PGHH,16794.95,18360.87,0.33,-6.87,NaN,-1.15,9.32,11.50,AV40,ATH,0,0,H,FMCG
65,HAVELLS,1768.50,1929.45,17.35,27.73,UP,-5.05,9.10,43.33,AV40,ATH,1,1,H,ELECTRICAL
18,BAJAJ-AUTO,9386.05,9961.75,23.42,74.36,UP,-2.49,6.13,106.42,AV40,ATH,1,1,H,AUTO
104,NAM-INDIA,635.20,671.25,28.84,16.56,UP,-2.56,5.68,111.30,AV40,ATH,1,0,H,FINANCE
60,GLAXO,2573.15,2700.70,28.14,33.56,UP,0.37,4.96,87.40,AV40,ATH,1,0,H,PHARMA


In [5]:
# av40n
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['AV40N'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Momentum,Today%,FTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
101,MOTILALOFS,532.20,1537.38,-22.31,-72.56,DOWN,-1.96,188.87,73.08,AV40N,ATH,1,1,M,FINANCE
85,JCHAC,1944.55,3219.25,50.27,NaN,UP,-2.73,65.55,106.68,AV40N,ATH,1,0,M,AC
157,VIPIND,463.95,740.05,-16.25,NaN,NaN,-1.82,59.51,2.44,AV40N,ATH,1,0,M,MISC
51,EQUITASBNK,89.58,114.55,-9.52,-23.59,NaN,-0.59,27.87,8.52,AV40N,ATH,1,1,M,BANKS
120,POLYCAB,6349.85,7227.33,16.40,45.78,UP,-3.09,13.82,64.47,AV40N,ATH,1,1,M,CABLES
44,DIXON,11267.20,12679.60,52.05,66.64,UP,-5.68,12.54,180.81,AV40N,ATH,1,1,M,IT
13,ASTRAZEN,6347.80,7104.85,16.30,4.95,UP,-1.62,11.93,68.94,AV40N,ATH,1,1,M,PHARMA
137,SYMPHONY,1094.75,1201.00,15.72,-6.85,UP,-4.38,9.71,33.06,AV40N,NTT,1,0,M,DURABLES
100,MCX,3824.15,4167.80,15.53,506.99,UP,-5.99,8.99,149.46,AV40N,ATH,1,0,M,MISC
34,CERA,8720.45,9483.65,10.62,18.00,NaN,-3.08,8.75,32.40,AV40N,ATH,1,1,M,CERAMICS


In [6]:
# avr
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['AVR'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Momentum,Today%,FTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
149,TV18BRDCST,40.83,211.63,-16.67,-902.42,DOWN,-2.76,418.32,3.89,AVR,ATH,1,0,M,ENTERTAINMENT
33,CENTRALBK,62.57,190.06,9.77,38.02,UP,-2.81,203.76,109.26,AVR,ATH,1,1,M,BANKS
11,ARVINDFASN,473.95,784.00,7.96,NaN,UP,-4.67,65.42,58.18,AVR,ATH,1,0,L,MISC
130,SHARDACROP,495.35,735.06,21.41,NaN,DOWN,-0.50,48.39,54.24,AVR,ATH,1,0,L,CHEMICALS
145,TIDEWATER,2258.90,3126.80,44.99,NaN,UP,-4.09,38.42,117.60,AVR,ATH,1,1,M,CHEMICALS
58,GEOJITFSL,111.55,153.30,41.20,NaN,UP,-4.21,37.43,145.27,AVR,ATH,1,1,L,FINANCE
4,AETHER,881.10,1165.95,1.98,58.16,DOWN,0.20,32.33,13.00,AVR,ATH,1,0,L,CHEMICALS
134,SRF,2342.60,2812.35,-1.45,56.53,NaN,-2.57,20.05,11.39,AVR,ATH,1,0,M,CHEMICALS
70,HINDPETRO,341.65,410.00,19.46,-35.48,UP,-4.47,20.01,117.74,AVR,BTT,1,1,M,REFINERIES
161,WSTCSTPAPR,655.40,782.95,-2.90,NaN,NaN,-2.71,19.46,43.58,AVR,ATH,1,0,M,PAPER
